In [ ]:
# MIT License
#Copyright (c) 2024 Mohamed Harhash  
#See the `LICENSE` file in the root directory for full license terms.
#You are free to use, modify, and distribute this software, provided that the original copyright notice and license terms are included. 
#Please cite this original code if you use it in your research or application.

#ICP-MS Automation Code: The code was developed by Mohamed Harhash during his Ph.D. at the Federal Institute of Hydrology, Koblenz, Germany 
# Code Author: Mohamed Harhash
## For any inquiries regards the code, I can be contacted at m.harhash@daad-alumni.de 


# The code was developed to automatically initiate the ICP Go software to operate the Agilent ICP-MS 7700 and ICP-QQQ-MS 8900 instruments or any other Agilent instrument compitable with ICP Go software
# It perform measurement sequences and automates the process by logging into the software, turning on the plasma, waiting for the warming time, selecting and creating batches, performing batch tuning, starting batches 
# and measurements, monitoring errors, sending emails about instrument status, and after measurements, signing out and closing the software
#please read manual for installation and more details.  


# import needed library
import time
import datetime
import logging
import smtplib
from email.mime.text import MIMEText
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

logging.basicConfig(filename='script.log', level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')


####                                                                    # 1-  open ICP Go software

### Opening ICPMS Go software

driver = webdriver.Chrome()
driver.get("http://172.40.30.72:5000") ### replace this IP address with the one provided by your software
driver.maximize_window()

#####                                                                     # 2- Logging into ICPMS Go #


wait = WebDriverWait(driver, 20)
username_input = wait.until(EC.element_to_be_clickable((By.NAME, "username")))

username_input.click()
username_input.send_keys("xxxx") ##### replace it with your own user name 
password_input = driver.find_element(By.NAME, "password")
password_input.send_keys("xxxx") #### replace it with your own password
login_button = driver.find_element(By.ID, "submit")
login_button.click()

## waiting 7 second to laod the page

time.sleep(7)

#driver.find_element(By.CSS_SELECTOR, "#ok .pe-ripple__mask").click() ### This was for handling a pop-up message that arises when logging into the ICPGo software, and you can disable it permanently from the MassHunter software if you need.





#####                                                                      # 3- Starting the plasma #



driver.find_element(By.CSS_SELECTOR, "#plasma .pe-ripple__mask").click()

time.sleep(2)
driver.find_element(By.CSS_SELECTOR, "#yes .pe-ripple__mask").click()




#####                                                                      # 4- Creating and managing new batches based on templates #

driver.find_element(By.CSS_SELECTOR, "svg").click()
driver.find_element(By.ID, "template").click()

##### determine the current date,time and day of the week

current_day_index = datetime.datetime.now().weekday()
current_time = datetime.datetime.now().time()
current_time_period = "morning" if current_time < datetime.datetime.strptime("12:00:00", "%H:%M:%S").time() else "evening"
day_names = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
selection = f"{day_names[current_day_index]} {current_time_period}"

##### click dropdown list for selecting proper batche for the current day and time 

time.sleep(4)
dropdown_element = driver.find_element(By.ID, "template")
dropdown_element.click()
dropdown = Select(dropdown_element)
dropdown.select_by_visible_text(selection)

#### Naming the created new batch as day name 'evening' or 'morning' and date

timestamp = datetime.datetime.now().strftime("%d_%m_%Y")
element_name = f"{selection}_{timestamp}"
print(element_name)
driver.find_element(By.NAME, "batchname").click()
time.sleep(5)
driver.find_element(By.NAME, "batchname").send_keys(element_name)
time.sleep(10)

###### Saving the batche 

driver.find_element(By.CSS_SELECTOR, "#savebatch .pe-ripple__mask").click()
time.sleep(10)

########                                                                    # 5- Add the created batch to queue #   

time.sleep(25)
xpath_selector = f'//*[@id="queuebutton_{element_name}"]/div'
element = driver.find_element(By.XPATH, xpath_selector)
element.click()

#######                                                                     # 6- Waiting warming time #

time.sleep(1900)

## Refersh the software page and handling all pop-up messages 

driver.refresh()
check_interval = 5

def handle_alert(driver, check_interval):
    try:
        yes_button = WebDriverWait(driver, check_interval).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#yes .pe-ripple__mask"))
        )
        yes_button.click()
        time.sleep(3)
        try:
            ok_button = driver.find_element(By.CSS_SELECTOR, "#ok .pe-ripple__mask")
            ok_button.click()
        except NoSuchElementException:
            print("The 'OK' button was not found. Continuing without clicking it.")
    except TimeoutException:
        print("The 'Yes' button was not found after refreshing. Continuing without clicking it.")

handle_alert(driver, check_interval)

def retry_login(driver, wait):
    try:
        username_element = wait.until(EC.element_to_be_clickable((By.NAME, "username")))
        username_element.click()
        username_element.send_keys("xxxx")  # replace xxxx with your own user name 
    except TimeoutException:
        print("The 'username' element was not found. Continuing without it.")
    try:
        password_element = wait.until(EC.element_to_be_clickable((By.NAME, "password")))
        password_element.click()
        password_element.send_keys("xxxxx") # replace xxxx with your own password 
    except TimeoutException:
        print("The 'password' element was not found. Continuing without it.")
    try:
        login_button = wait.until(EC.element_to_be_clickable((By.ID, "submit")))
        login_button.click()
    except TimeoutException:
        print("The 'login' button was not found. Continuing without attempting to log in.")

retry_login(driver, wait)
handle_alert(driver, check_interval)


def click_with_retry(driver, button_selector, retry_interval=40):
    while True:
        try:
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, button_selector)))
            button.click()
            break
        except (TimeoutException, NoSuchElementException):
            print(f"Button {button_selector} not found or not clickable. Retrying in {retry_interval} seconds...")
            time.sleep(retry_interval)

            
            
###                                                                                       # 7- Run the queue and start tuning and the batch

run_button_selector = "#run .pe-ripple__mask"
queue_button_selector = "#queue .pe-ripple__mask"

click_with_retry(driver, run_button_selector)
time.sleep(20)
click_with_retry(driver, queue_button_selector)

#####                                                                                      # 8- Waiting tuning of the batch till finish


time.sleep(180)


#####                                                                                      # 9- check the error and send an email about the instrument status # 

try:
    instrument_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#instrument .pe-ripple__mask"))
    )
    instrument_button.click()
except Exception as e:
    print(f"Failed to click the instrument button: {e}")

error_message_selector = "#errorLog > div"
try:
    error_message_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, error_message_selector))
    )
    error_message_text = error_message_element.text
    if error_message_text:
        gmail_email = "your email @gmail.com"    ########## your E-mail 
        gmail_password = "xxx xxx xxx xxx"            #### access code of your E-mail obtained from you eamil setting ## your-app-password
        recipient_email = " recipient-email@example.com "                 ######## here you can use your E-mail 
        subject = "Error Alert_ICPMS_Monde"                  ######### you can change it to the name you need 
        body = f"Error Message: {error_message_text}"

        msg = MIMEText(body)
        msg["Subject"] = subject
        msg["From"] = gmail_email
        msg["To"] = recipient_email

        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(gmail_email, gmail_password)
        server.sendmail(gmail_email, recipient_email, msg.as_string())
        server.quit()
except Exception as e:
    print(f"An error occurred: {e}")

print("Email sent successfully.")



################                                                                           # 10- Waiting time for the measurement (around 4 hours) and it is more than our measurment time to ensure enough time after measurment ### 

driver.find_element(By.CSS_SELECTOR, "#queue .pe-ripple__mask").click()

time.sleep(16000)


###################                                                                         # 11-  Sing out  nad close ICP Go sodftware                                                   #


try:
    logout_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#login .pe-ripple__mask"))
    )
    logout_button.click()
except TimeoutException:
    print("Logout button not clickable. Continuing without clicking.")
try:
    ok_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#ok .pe-ripple__mask"))
    )
    ok_button.click()
except TimeoutException:
    print("Ok button not clickable. Continuing without clicking.")


time.sleep(5)
driver.quit()
